In [1]:
from load import *
from torch import Tensor, nn
import torch
from model_base import Model, ModelBase, Linear

ROOT = "./Datas/Week 8"

Q = 1.60217663e-19

In [4]:
class Sequential(Model):
    def __init__(self, *f):
        self.fs = f

    def forward(self, x):
        for model in self.fs:
            x = model(x)
        return x
    
    def _model_children(self):
        for i, model in enumerate(self.fs):
            assert isinstance(model, Model)
            yield str(i), model
    
    def _deserialize(self, state: dict):
        for k, v in state.items():
            if k in ("_init_=args", "_init_=kwargs"):
                continue
            self.fs[int(k)]._deserialize(v)
        return self

c = Sequential(
    Linear(100, 10),
    Linear(10, 5),
    Linear(5, 2)
)

t = torch.randn(500, 100)
y = c(t)
print(y.shape)

torch.Size([500, 2])


In [5]:
c.save("./model.hlpt")

In [6]:
d = Sequential.load("./model.hlpt")

In [7]:
print(d)

Sequential
Model summary:
Sequential (Trainable: 1077, Other: 0)
- Linear (Trainable: 1010, Other: 0)
- Linear (Trainable: 55, Other: 0)
- Linear (Trainable: 12, Other: 0)



In [8]:
z = d(t)
torch.all(y == z)

tensor(True)